In [1]:
from carbon.helpers.cryptocompare import CryptoCompare
import pandas as pd
from matplotlib import pyplot as plt
import os

print( "{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CryptoCompare))

CryptoCompare v1.0 (27/Jan/2023)


# Crypto Compare

https://min-api.cryptocompare.com/documentation

In [2]:
CC, CCA = CryptoCompare(), CryptoCompare(apikey=True)
print("[CryptoCompare] key digest", CC.keydigest[:4], CCA.keydigest[:4])

[CryptoCompare] key digest 86a6 0000


In [3]:
#CC.query_ratelimit()
#CCA.query_ratelimit()

## Get daily data

In [4]:
CCNE = CryptoCompare(raiseonerror=False)

In [5]:
coins ="ETH, BTC, BNB, XRP, ADA, DOGE, MATIC, SOL, OKB, DOT, LTC, TRX, AVAX, ATOM"
usdstables = "USDT, USDC, UST, BUSD"
ccies = "USD, BTC, ETH"

In [18]:
# fsyms = CC.unjoin(usdstables)
# tsyms = ["USD",]
# results = {
#     (fsym, tsym): CCNE.query_dailypair(fsym=fsym, tsym=tsym)
#     for fsym in fsyms
#     for tsym in tsyms
# }
# df = pd.concat(results, axis=1)
# df.to_pickle(CC.datafn("USD-STABLES-RAW.pickle"))

# df = pd.read_pickle(CC.datafn("USD-STABLES-RAW.pickle"))
# df = CC.reformat_raw_df(df)
# df.to_pickle(CC.datafn("USD-STABLES.pickle"))
# df = pd.read_pickle(CC.datafn("USD-STABLES.pickle"))
# df

,USDT/USD,UST/USD,USDC/USD,BUSD/USD
datetime,,,,
2017-08-06,1.0,0.0000,0.0000,0.0000
2017-08-07,1.0,0.0000,0.0000,0.0000
2017-08-08,1.0,0.0000,0.0000,0.0000
2017-08-09,1.0,0.0000,0.0000,0.0000
2017-08-10,1.0,0.0000,0.0000,0.0000
...,...,...,...,...
2023-01-23,1.0,0.0213,0.9998,1.0000
2023-01-24,1.0,0.0213,1.0000,1.0000
2023-01-25,1.0,0.0213,1.0000,1.0000


In [7]:
#pd.read_pickle("cryptocompare/USD-STABLES-RAW.pickle")

In [17]:
# fsyms = CC.unjoin(coins)
# tsyms = ["USD",]
# results = {
#     (fsym, tsym): CCNE.query_dailypair(fsym=fsym, tsym=tsym)
#     for fsym in fsyms
#     for tsym in tsyms
# }
# df = pd.concat(results, axis=1)
# df.to_pickle(CC.datafn("USD-COINS-RAW.pickle"))

# df = pd.read_pickle(CC.datafn("USD-COINS-RAW.pickle"))
# df = CC.reformat_raw_df(df)
# df.to_pickle(CC.datafn("USD-COINS.pickle"))
# df = pd.read_pickle(CC.datafn("USD-COINS.pickle"))
# df

,OKB/USD,BNB/USD,DOT/USD,TRX/USD,BTC/USD,AVAX/USD,ATOM/USD,ADA/USD,SOL/USD,LTC/USD,XRP/USD,DOGE/USD,MATIC/USD,ETH/USD
datetime,,,,,,,,,,,,,,
2017-08-06,0.00,0.00,0.000,0.00000,3262.80,0.00,0.00,0.0000,0.00,46.24,0.1848,0.002011,0.0000,253.09
2017-08-07,0.00,0.00,0.000,0.00000,3232.03,0.00,0.00,0.0000,0.00,45.56,0.1802,0.001935,0.0000,264.56
2017-08-08,0.00,0.00,0.000,0.00000,3401.91,0.00,0.00,0.0000,0.00,46.11,0.1782,0.001881,0.0000,269.94
2017-08-09,0.00,0.00,0.000,0.00000,3429.38,0.00,0.00,0.0000,0.00,49.13,0.1967,0.001965,0.0000,296.51
2017-08-10,0.00,0.00,0.000,0.00000,3348.79,0.00,0.00,0.0000,0.00,47.78,0.1841,0.001914,0.0000,295.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-23,36.64,302.76,6.206,0.06163,22714.99,17.36,13.31,0.3765,24.22,87.60,0.4007,0.088230,0.9942,1627.79
2023-01-24,35.93,305.24,6.567,0.06273,22917.47,17.98,13.05,0.3750,24.35,89.91,0.4238,0.088480,0.9936,1626.47
2023-01-25,34.93,300.46,6.185,0.06016,22637.80,17.00,12.43,0.3589,22.77,87.25,0.4077,0.083890,0.9555,1556.12


In [9]:
#pd.read_pickle("cryptocompare/USD-COINS-RAW.pickle")

In [19]:
# fsyms = CC.unjoin(coins)
# tsyms = ["BTC",]
# results = {
#     (fsym, tsym): CCNE.query_dailypair(fsym=fsym, tsym=tsym)
#     for fsym in fsyms
#     for tsym in tsyms
# }
# df = pd.concat(results, axis=1)
# df.to_pickle(CC.datafn("BTC-COINS-RAW.pickle"))

# df = pd.read_pickle(CC.datafn("BTC-COINS-RAW.pickle"))
# df = CC.reformat_raw_df(df)
# df.to_pickle(CC.datafn("BTC-COINS.pickle"))
# df = pd.read_pickle(CC.datafn("BTC-COINS.pickle"))
# df

,OKB/BTC,BNB/BTC,DOT/BTC,BTC/BTC,AVAX/BTC,ADA/BTC,SOL/BTC,TRX/BTC,XRP/BTC,DOGE/BTC,MATIC/BTC,ETH/BTC,LTC/BTC,ATOM/BTC
datetime,,,,,,,,,,,,,,
2017-08-06,0.000000,0.00000,0.000000,1,0.000000,0.000000,0.000000,0.000000,0.000057,6.300000e-07,0.000000,0.07785,0.014180,0.000000
2017-08-07,0.000000,0.00000,0.000000,1,0.000000,0.000000,0.000000,0.000000,0.000056,6.000000e-07,0.000000,0.08197,0.014050,0.000000
2017-08-08,0.000000,0.00000,0.000000,1,0.000000,0.000000,0.000000,0.000000,0.000052,5.700000e-07,0.000000,0.07946,0.013500,0.000000
2017-08-09,0.000000,0.00000,0.000000,1,0.000000,0.000000,0.000000,0.000000,0.000057,5.800000e-07,0.000000,0.08662,0.014320,0.000000
2017-08-10,0.000000,0.00000,0.000000,1,0.000000,0.000000,0.000000,0.000000,0.000055,5.800000e-07,0.000000,0.08829,0.014250,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-23,0.001613,0.01334,0.000273,1,0.000763,0.000017,0.001066,0.000003,0.000018,3.890000e-06,0.000044,0.07164,0.003856,0.000587
2023-01-24,0.001568,0.01332,0.000286,1,0.000784,0.000016,0.001061,0.000003,0.000018,3.860000e-06,0.000043,0.07097,0.003922,0.000570
2023-01-25,0.001543,0.01327,0.000273,1,0.000751,0.000016,0.001006,0.000003,0.000018,3.710000e-06,0.000042,0.06875,0.003853,0.000550
